In [1]:
import time
time_start = time.time()
import os
import torch
import gc
import math
ON_KAGGLE = os.path.exists('/kaggle/input')
ENABLE_SUBMISSION = True
GENERATE_WHL = False
os.environ["ON_KAGGLE"] = str(ON_KAGGLE)
env = None
os.environ["USE_VLLM"] = "False"
# Clear cache, helps in re-running the notebook
gc.collect()
torch.cuda.empty_cache()
# This is related to T4 GPU on collab
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
# Install required packages if on Kaggle
if ON_KAGGLE:
    !pip install -U /kaggle/input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/omegaconf222py3/antlr4_python3_runtime-4.9.3-py3-none-any.whl -qq
    !pip install -U /kaggle/input/omegaconf222py3/omegaconf-2.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/hydracore120py3/hydra_core-1.2.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/parglare-0-18-0/parglare-0.18.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/rank-bm25-whl/rank_bm25-0.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/huggingface-trl-package/shtab-1.7.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/huggingface-trl-package/tyro-0.7.3-py3-none-any.whl -qq
    !pip install -U /kaggle/input/peft-0-11-1/peft-0.11.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/huggingface-trl-package/trl-0.7.11-py3-none-any.whl -qq
    !pip install -U /kaggle/input/amino-gaz-whl/aimo_gaz-0.0.1-py3-none-any.whl -qq --no-deps --force-reinstall
else:
    !pip install -r requirements.txt
    !pip install -e . --force-reinstall
if GENERATE_WHL:
    !pip install --upgrade build
    !python -m build

In [ ]:
import time
from clever_prover.solver.solver_and_tool_config import parse_solver_or_tool_config
from clever_prover.utils.log_utils import setup_logger
from hydra import compose, initialize

config_path = '/kaggle/input/aimo-gaz-configs/configs' if ON_KAGGLE else 'src/clever_prover/configs'
config_name = 'coordination_solver_config.yaml'

# Setup the logger
if "CLEVER_PROVER_ROOT" not in os.environ:
    os.environ["CLEVER_PROVER_ROOT"] = '/kaggle/working' if ON_KAGGLE else 'src/clever_prover'
log_dir_name = "logs" if ON_KAGGLE else '.logs'
root_dir = os.environ["CLEVER_PROVER_ROOT"]
# Check if the configs exists in the root directory
if not os.path.exists(os.path.join(root_dir, 'configs')):
    os.makedirs(os.path.join(root_dir, 'configs'))
# Copy the configs to the root directory
os.system(f'cp -r {config_path}/* {os.path.join(root_dir, "configs")}')
# Create the logs directory
if not os.path.exists(os.path.join(root_dir, log_dir_name)):
    os.makedirs(os.path.join(root_dir, log_dir_name), exist_ok=True)
time_str = time.strftime("%Y-%m-%d_%H-%M-%S")
os.makedirs(os.path.join(root_dir, log_dir_name, time_str), exist_ok=True)
logger = setup_logger("clever_prover", os.path.join(root_dir, log_dir_name, time_str, 'clever_prover.log'))
logger.info(f"Using config file: {config_name}")
with initialize(version_base="1.2", config_path="configs/", job_name="clever_prover"):
    cfg = compose(config_name=config_name) # Load the config file
    solver_config = parse_solver_or_tool_config(cfg)
solver = solver_config.get_solver_or_tool(logger=logger)
assert solver is not None, "Solver not initialized"
    

In [ ]:
# Set up the evaluation API
if ENABLE_SUBMISSION and ON_KAGGLE:
    import aimo
    if env is None:
        env = aimo.make_env()
    iter_test = env.iter_test()

In [ ]:
if ENABLE_SUBMISSION and ON_KAGGLE:
    prep_time = math.ceil(time.time() - time_start)
    total_time_left = 9 * 60 * 60 - prep_time - 700 # 9 hours in seconds - time taken for preparation and grace time of 100s
    test_problems = set(["What is $1-1$?", "What is $0\\times10$?", "Solve $4+x=4$ for $x$."])
    reduced_time_for_test_problems = 50
    with solver:
        problems_attempted = 0
        total_problems = 50
        # Iterate through the test set and use the model make predictions
        for test, sample_submission in iter_test:
            start_timer = time.time()
            print("Problem statement:\n")
            problem_statement = str(test['problem'][0])
            print(problem_statement)
            if problem_statement in test_problems:
                time_allowed_to_solve = reduced_time_for_test_problems
            else:
                time_allowed_to_solve = total_time_left // (total_problems - problems_attempted)
            print(f"Time allowed to solve {time_allowed_to_solve}")
            sample_submission['answer'] = solver.solve(problem_statement, time_allowed = time_allowed_to_solve)
            env.predict(sample_submission)
            print("Submission:\n")
            print(sample_submission, '\n')
            print("="*50)
            time_to_attempt = math.ceil(time.time() - start_timer)
            total_time_left -= time_to_attempt
            problems_attempted += 1
            if total_time_left <= 0:
                break